In [1]:
import torch
from torch import nn
from torch.utils.data import TensorDataset, DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from Adam import Adam
from timeit import default_timer
import operator
from functools import reduce
from functools import partial
from normal import UnitGaussianNormalizer
from matplotlib import cm
import scipy.io
from utilities3 import LpLoss
from deeponet import DeepONet2D
from gru2 import GRUModel

In [2]:
x = np.loadtxt("xAlpha.dat", np.float32)
y = np.loadtxt("y.dat", np.float32)
x = x.reshape(x.shape[0], x.shape[1]//7, 7)
y = y.reshape(y.shape[0], y.shape[1]//5, 5)
grid = np.linspace(0, 20, 2000, dtype=np.float32).reshape((2000, 1))

print(x.shape)
print(y.shape)
print(grid.shape)

(1000, 2000, 7)
(1000, 2000, 5)
(2000, 1)


In [3]:
# Define a sequential torch network for batch and trunk. Can use COV2D which we will show later
branch = nn.Sequential(
    GRUModel(200, 7, 500))

trunk = nn.Sequential(
          nn.Linear(1, 64),
          nn.ReLU(),
          nn.Linear(64,128),
          nn.ReLU(),
          nn.Linear(128, 500),
          nn.ReLU(),
        )

In [4]:
# Parameters
epochs = 1000
ntrain = 900
ntest = 100
batch_size = 20
gamma = 0.5
learning_rate = 0.001
step_size= 100

In [5]:
# Create train/test splits
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1, random_state=1)
x_train = torch.from_numpy(x_train).cuda()
y_train = torch.from_numpy(y_train).cuda()
x_test = torch.from_numpy(x_test).cuda()
y_test = torch.from_numpy(y_test).cuda()

grid_repeated = []
for i in range(batch_size):
    grid_repeated.append(grid)
grid_repeated = torch.from_numpy(np.array(grid_repeated)).cuda()

trainData = DataLoader(TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
testData = DataLoader(TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

In [6]:
def count_params(model):
    c = 0
    for p in list(model.parameters()):
        c += reduce(operator.mul, 
                    list(p.size()+(2,) if p.is_complex() else p.size()))
    return c

In [7]:
model = DeepONet2D(branch, trunk, 5).cuda()
print(count_params(model))

298848


In [8]:
optimizer = Adam(model.parameters(), lr=learning_rate, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=step_size, gamma=gamma)

In [9]:
loss = LpLoss()
train_mseArr = []
test_mseArr = []
for ep in range(epochs):
    model.train()
    t1 = default_timer()
    train_mse = 0
    for x, y in trainData:
        x, y = x.cuda(), y.cuda()
        optimizer.zero_grad()
        
        out = model(x, grid_repeated)
        mse = loss(out, y)
        mse.backward()
        
        optimizer.step()
        train_mse += mse.item()
        
    scheduler.step()
    model.eval()
    test_mse = 0
    with torch.no_grad():
        for x, y in testData:
            x, y = x.cuda(), y.cuda()
            
            out = model(x, grid_repeated)
            test_mse += loss(out, y).item()
            
    train_mse /= len(trainData)
    test_mse /= len(testData)
    
    train_mseArr.append(train_mse)
    test_mseArr.append(test_mse)
    
    t2 = default_timer()
    if ep%50 == 0:
        print(ep, t2-t1, train_mse, test_mse)

0 2.3844285397790372 0.3654269943634669 0.08371993154287338
50 2.20546884695068 0.0184387339072095 0.008610419556498527


KeyboardInterrupt: 

In [ ]:
plt.plot(train_mseArr, label="Train")
plt.plot(test_mseArr, label="Test")
plt.yscale("log")
plt.title("DeepONet Loss")
plt.xlabel("Epoch Number")
plt.ylabel(r"$L_2$ Loss")
plt.legend()
plt.savefig("deeponet.eps")

In [ ]:
# Show some examples at the boundry
index = 0
fig, ax = plt.subplots(1, 5, figsize=(20, 5))
with torch.no_grad():
    for x, y in testData:
        x, y = x.cuda(), y.cuda()   
        out = model(x, grid_repeated)
        l1 = ax[index].plot(out[0,:, 2].detach().cpu(), label="Neural Network Model")
        l2 = ax[index].plot(y[0, :, 2].detach().cpu(), label="Statistical Parameter Estimator")
        ax[index].set_xlabel("t(s)")
        ax[index].set_ylabel("$\hat{\theta}(0, t)$")
        index+=1
#fig.legend([l1, l2], ["Neural Network Model", "Statistical Parameter Estimator"])
#fig.show()

In [ ]:
torch.save(model.state_dict(), 'deeponet1')

In [8]:
model.load_state_dict(torch.load('deeponet1'))

<All keys matched successfully>

In [11]:
with torch.no_grad():
    for x, y in testData:
        x, y = x.cuda(), y.cuda()   
        startTime = default_timer()
        out = model(x, grid_repeated)
        endTime = default_timer()

In [12]:
print(endTime-startTime)

0.02297679567709565
